# Policy for Amazon Bedrock AgentCore - 엔드투엔드 튜토리얼

## 개요

이 노트북은 **Policy for Amazon Bedrock AgentCore**를 구현하는 방법을 보여줍니다. 이 기능은 Cedar policy와 JWT token claim을 사용하여 Amazon Bedrock AgentCore Gateway에서 AI agent의 tool 호출에 대한 세밀한 접근 제어를 가능하게 합니다.

### 학습 내용

- Amazon Cognito를 구성하여 JWT token에 커스텀 claim을 추가하는 방법
- Amazon Bedrock AgentCore Identity를 통해 제공된 JWT claim을 검증하는 Cedar policy를 생성하는 방법
- 속성 기반 접근 제어(ABAC) 패턴을 구현하는 방법
- 연결된 Amazon Bedrock AgentCore Gateway에서 다양한 claim 시나리오로 policy 적용을 테스트하고 검증하는 방법

### 주요 개념

**Policy for Amazon Bedrock AgentCore**: JWT token claim을 principal 속성으로 사용하여 세밀한 policy에 대한 접근 요청을 평가하는 Cedar 기반 policy engine입니다.

**Amazon Bedrock AgentCore Identity**: Amazon Cognito 또는 OAuth를 지원하는 다른 Identity Provider(IdP)와 통합하여 요청을 인증하고 policy 평가를 위한 JWT claim을 추출합니다.

**Amazon Bedrock AgentCore Gateway**: 개발자가 MCP를 사용하여 대규모로 tool을 구축, 배포, 검색 및 연결할 수 있는 쉽고 안전한 방법입니다.

**[Cedar Policies](https://www.cedarpolicy.com/en)**: 누가(principal) 어떤 리소스에 대해 무엇을(action) 수행할 수 있는지를 선택적 조건과 함께 정의하는 선언적 policy입니다. 자세한 내용은 [cedarpolicy.com](https://www.cedarpolicy.com/en)에서 확인하세요.

### 샘플 아키텍처

```
                                ┌───────────────────────┐
                                │  Policy for AgentCore │
                                │  (Cedar Policies)     │
                                │                       │
                                │  Evaluates:           │
                                │  - principal tags     │
                                │  - context.input      │
                                │  - resource           │
                                └───────────┬───────────┘
                                            │ attached
                                            ▼
┌─────────────────┐             ┌───────────────────────┐             ┌─────────────┐
│   Amazon        │  JWT Token  │  Amazon Bedrock       │             │   Lambda    │
│   Cognito       │────────────▶│  AgentCore Gateway    │────────────▶│   Target    │
│   + AWS Lambda  │  with       │                       │  if ALLOWED │   (Tool)    │
└─────────────────┘  claims     └───────────────────────┘             └─────────────┘
```

### 사전 요구 사항

- 적절한 IAM 권한이 있는 AWS 계정
- OAuth authorizer로 이미 구성된 Amazon Bedrock AgentCore Gateway
- 앱 클라이언트(M2M)가 있는 Amazon Cognito User Pool
- boto3 및 requests가 설치된 Python 3.8+
- 구성된 AWS 자격 증명

---

## Part 1: 설정 및 구성

먼저 필요한 종속성을 설치하고 구성을 초기화하겠습니다.

In [ ]:
# 필요한 패키지 설치
%pip install -r requirements.txt

In [ ]:
import json
import os
import time
import base64
import zipfile
import tempfile
from pathlib import Path
from typing import Dict, Any, Optional, List

import boto3  # AWS SDK
import requests  # HTTP 요청 라이브러리
from botocore.exceptions import ClientError

print("✓ Libraries imported successfully")

### Step 1.1: 구성 로드 또는 생성

이 튜토리얼에는 Amazon Bedrock AgentCore Gateway 세부 정보와 Amazon Cognito 클라이언트 정보가 포함된 `gateway_config.json` 파일이 필요합니다. 이러한 리소스가 아직 없는 경우 동일한 폴더에 제공된 유틸리티 스크립트 `setup-gateway.py`를 실행하여 생성할 수 있습니다.

#### 예상 구성 구조

```json
{
  "gateway_url": "https://<gateway-id>.gateway.policy-registry.<region>.amazonaws.com/mcp",
  "gateway_id": "<gateway-id>",
  "gateway_arn": "arn:aws:policy-registry:<region>:<account-id>:gateway/<gateway-id>",
  "region": "<region>",
  "client_info": {
    "client_id": "<cognito-app-client-id>",
    "client_secret": "<cognito-app-client-secret>",
    "user_pool_id": "<region>_<pool-id>",
    "token_endpoint": "https://<domain>.auth.<region>.amazoncognito.com/oauth2/token"
  },
  "policy_engine_id": "<optional-policy-engine-id>"
}
```

#### 사전 요구 사항 설정

이러한 리소스가 아직 구성되지 않은 경우 이 노트북에서 직접 설정 스크립트를 실행할 수 있습니다. 스크립트는 다음을 수행합니다:
- OAuth authorization이 있는 Amazon Bedrock AgentCore Gateway 생성
- 테스트용 샘플 Refund Lambda 함수 생성
- Lambda를 Gateway에 target으로 연결
- 구성을 `gateway_config.json`에 저장

아래 셀을 실행하여 설정 스크립트를 실행하세요:


In [ ]:
# Gateway 설정을 위한 region 구성 (필요에 따라 변경)
SETUP_REGION = "us-east-1"  # 원하는 region으로 변경

# 선택 사항: trust relationship이 구성된 IAM role ARN 지정
# 자동으로 새 role을 생성하려면 None으로 유지
SETUP_ROLE_ARN = None  # 예: "arn:aws:iam::123456789012:role/MyGatewayRole"

In [ ]:
# Gateway와 Lambda target을 자동으로 설정하려면 이 셀을 실행
import subprocess
import sys

cmd = [sys.executable, "setup-gateway.py", "--region", SETUP_REGION]
if SETUP_ROLE_ARN:
    cmd.extend(["--role-arn", SETUP_ROLE_ARN])

result = subprocess.run(cmd, capture_output=False, text=True)
if result.returncode != 0:
    print(f"Setup failed with return code: {result.returncode}")

또는 다음 가이드를 따라 리소스를 수동으로 설정할 수 있습니다:

1. **Amazon Bedrock AgentCore Gateway**: [Gateway 빠른 시작 가이드](https://docs.aws.amazon.com/policy-registry/latest/devguide/gateway.html)
2. **Gateway에 Target 추가**: [Gateway Target 문서](https://docs.aws.amazon.com/policy-registry/latest/devguide/gateway-targets.html)
3. **Amazon Cognito User Pool**: [Amazon Cognito 개발자 가이드](https://docs.aws.amazon.com/cognito/latest/developerguide/cognito-user-pools.html)


In [ ]:
# gateway_config.json 템플릿
CONFIG_TEMPLATE = {
    "gateway_url": "https://<gateway-id>.gateway.policy-registry.<region>.amazonaws.com/mcp",
    "gateway_id": "<gateway-id>",
    "gateway_arn": "arn:aws:policy-registry:<region>:<account-id>:gateway/<gateway-id>",
    "region": "<region>",
    "client_info": {
        "client_id": "<cognito-app-client-id>",
        "client_secret": "<cognito-app-client-secret>",
        "user_pool_id": "<region>_<pool-id>",
        "token_endpoint": "https://<domain>.auth.<region>.amazoncognito.com/oauth2/token",
    },
}


def load_or_create_gateway_config() -> Dict[str, Any]:
    """
    gateway_config.json에서 gateway 구성을 로드.
    파일이 없으면 사용자가 작성할 템플릿 생성.
    """
    config_path = Path.cwd() / "gateway_config.json"

    if not config_path.exists():
        # 템플릿 파일 생성
        with open(config_path, "w", encoding="utf-8") as f:
            json.dump(CONFIG_TEMPLATE, f, indent=2)

        print("⚠️  gateway_config.json not found!")
        print(f"\n✓ Created template at: {config_path}")
        print("\nPlease fill in the configuration with your actual values:")
        print("  1. Set up an Amazon Bedrock AgentCore Gateway")
        print("  2. Create an Amazon Cognito User Pool with an app client (M2M)")
        print("  3. Update gateway_config.json with your resource details")
        print("  4. Re-run this cell")
        print("\nDocumentation:")
        print(
            "  - Gateway: https://docs.aws.amazon.com/policy-registry/latest/devguide/gateway.html"
        )
        print(
            "  - Cognito: https://docs.aws.amazon.com/cognito/latest/developerguide/cognito-user-pools.html"
        )
        raise FileNotFoundError(
            "Please configure gateway_config.json and re-run this cell."
        )

    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    # 필수 필드 검증
    required_fields = [
        "gateway_url",
        "gateway_id",
        "gateway_arn",
        "region",
        "client_info",
    ]
    missing = [
        f for f in required_fields if f not in config or "<" in str(config.get(f, ""))
    ]

    if missing:
        print("⚠️  Configuration incomplete!")
        print(f"   Please update these fields in gateway_config.json: {missing}")
        raise ValueError(f"Missing or placeholder values in config: {missing}")

    # client_info 필드 검증
    client_info_fields = [
        "client_id",
        "client_secret",
        "user_pool_id",
        "token_endpoint",
    ]
    client_info = config.get("client_info", {})
    missing_client = [
        f
        for f in client_info_fields
        if f not in client_info or "<" in str(client_info.get(f, ""))
    ]

    if missing_client:
        print("⚠️  Client info incomplete!")
        print(f"   Please update client_info fields: {missing_client}")
        raise ValueError(
            f"Missing or placeholder values in client_info: {missing_client}"
        )

    return config


# 구성 로드
CONFIG = load_or_create_gateway_config()

# 주요 값 추출
REGION = CONFIG["region"]
GATEWAY_URL = CONFIG["gateway_url"]
GATEWAY_ID = CONFIG["gateway_id"]
GATEWAY_ARN = CONFIG["gateway_arn"]
USER_POOL_ID = CONFIG["client_info"]["user_pool_id"]
CLIENT_ID = CONFIG["client_info"]["client_id"]
CLIENT_SECRET = CONFIG["client_info"]["client_secret"]
TOKEN_ENDPOINT = CONFIG["client_info"]["token_endpoint"]
POLICY_ENGINE_ID = CONFIG.get("policy_engine_id")

print("✓ Configuration loaded successfully")
print(f"  Region: {REGION}")
print(f"  Gateway ID: {GATEWAY_ID}")
print(f"  Gateway URL: {GATEWAY_URL}")
print(f"  User Pool ID: {USER_POOL_ID}")
print(f"  Policy Engine ID: {POLICY_ENGINE_ID or 'Not configured yet'}")

### Step 1.2: AWS Client 초기화

이 튜토리얼에 필요한 boto3 client를 생성합니다.

In [ ]:
# AWS client 초기화
session = boto3.Session(region_name=REGION)

lambda_client = session.client("lambda")
cognito_client = session.client("cognito-idp")
iam_client = session.client("iam")
sts_client = session.client("sts")

# Policy Engine과 Cedar policy 관리를 위한 AgentCore Control client
policy_client = session.client("bedrock-agentcore-control", region_name=REGION)

# 현재 계정 정보 가져오기
ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS clients initialized")
print(f"  Account ID: {ACCOUNT_ID}")
print(f"  Region: {REGION}")

### Step 1.3: Gateway Authorizer 구성 검증

이 단계는 gateway의 JWT authorizer가 Cognito access token에 대해 올바르게 구성되었는지 확인합니다.

**중요**: Amazon Cognito access token에는 `aud`(audience) claim이 포함되지 않습니다. gateway에 `allowedAudience`가 구성되어 있으면 token 검증이 401 오류와 함께 실패합니다. 이 단계는 필요한 경우 구성을 확인하고 수정합니다.

In [ ]:
# Gateway 구성 관리를 위한 gateway control client
gateway_control_client = session.client("bedrock-agentcore-control", region_name=REGION)


def get_gateway_details() -> Dict[str, Any]:
    """현재 gateway 세부 정보 가져오기."""
    return gateway_control_client.get_gateway(gatewayIdentifier=GATEWAY_ID)


def wait_for_gateway_ready(max_wait: int = 300, poll_interval: int = 5) -> bool:
    """Gateway가 READY 상태에 도달할 때까지 대기."""
    terminal_states = {"READY", "FAILED", "UPDATE_UNSUCCESSFUL"}
    start_time = time.time()

    while time.time() - start_time < max_wait:
        gateway = get_gateway_details()
        status = gateway.get("status", "UNKNOWN")
        print(f"  Gateway status: {status}")

        if status == "READY":
            return True
        if status in terminal_states:
            print(f"  ✗ Gateway reached terminal state: {status}")
            return False

        time.sleep(poll_interval)

    print("  ✗ Timeout waiting for gateway")
    return False


def validate_and_fix_gateway_authorizer() -> bool:
    """
    Gateway authorizer 구성을 검증하고 필요시 수정.

    Cognito access token에는 'aud' claim이 없으므로 allowedAudience를
    설정하지 않아야 함. 그렇지 않으면 gateway가 유효한 token을 거부함.

    Returns:
        구성이 유효하거나 성공적으로 수정된 경우 True
    """
    print("\nValidating Gateway Authorizer Configuration")
    print("=" * 70)

    gw = get_gateway_details()
    jwt_config = gw.get("authorizerConfiguration", {}).get("customJWTAuthorizer", {})

    # 현재 구성 확인
    discovery_url = jwt_config.get("discoveryUrl")
    allowed_clients = jwt_config.get("allowedClients", [])
    allowed_audience = jwt_config.get("allowedAudience", [])
    allowed_scopes = jwt_config.get("allowedScopes", [])

    print(f"  Discovery URL: {discovery_url or 'NOT SET'}")
    print(f"  Allowed Clients: {allowed_clients}")
    print(f"  Allowed Audience: {allowed_audience}")
    print(f"  Allowed Scopes: {allowed_scopes}")

    # 예상 discovery URL 구성
    expected_discovery_url = f"https://cognito-idp.{REGION}.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration"

    # 구성 수정 필요 여부 확인
    needs_fix = False
    reasons = []

    if not discovery_url:
        needs_fix = True
        reasons.append("Discovery URL not set")
    elif discovery_url != expected_discovery_url:
        needs_fix = True
        reasons.append("Discovery URL mismatch")

    if CLIENT_ID not in allowed_clients:
        needs_fix = True
        reasons.append(f"Client ID {CLIENT_ID} not in allowed clients")

    # Cognito access token에는 'aud' claim이 없음 - allowedAudience는 비어 있어야 함
    if allowed_audience:
        needs_fix = True
        reasons.append(
            "allowedAudience is set but Cognito access tokens don't have 'aud' claim"
        )

    if not needs_fix:
        print("\n✓ Gateway authorizer configuration is valid")
        return True

    print("\n⚠️  Configuration needs fixing:")
    for reason in reasons:
        print(f"   - {reason}")

    # 구성 수정
    print("\n⏳ Updating gateway authorizer configuration...")

    # 구성에서 scope 가져오기 (있는 경우)
    scope = CONFIG.get("client_info", {}).get("scope", "")

    new_auth_config = {
        "customJWTAuthorizer": {
            "discoveryUrl": expected_discovery_url,
            "allowedClients": [CLIENT_ID],
            # allowedAudience 설정하지 않음 - Cognito access token에는 'aud' claim이 없음
        }
    }

    # scope가 구성된 경우 추가
    if scope:
        new_auth_config["customJWTAuthorizer"]["allowedScopes"] = [scope]

    try:
        gateway_control_client.update_gateway(
            gatewayIdentifier=GATEWAY_ID,
            name=gw.get("name"),
            roleArn=gw.get("roleArn"),
            protocolType=gw.get("protocolType", "MCP"),
            authorizerType="CUSTOM_JWT",
            authorizerConfiguration=new_auth_config,
            policyEngineConfiguration=gw.get("policyEngineConfiguration", {}),
        )

        print("\n⏳ Waiting for gateway to become READY...")
        if wait_for_gateway_ready():
            print("\n✓ Gateway authorizer configuration fixed successfully")
            return True
        else:
            print("\n✗ Gateway did not reach READY state")
            return False

    except ClientError as e:
        print(f"\n✗ Error updating gateway: {e}")
        return False


# 필요시 gateway authorizer 검증 및 수정
validate_and_fix_gateway_authorizer()

---

## Part 2: 헬퍼 함수

이러한 유틸리티 함수는 튜토리얼 전체에서 token 관리, API 호출 및 응답 분석에 사용됩니다.

In [ ]:
def get_bearer_token() -> str:
    """
    OAuth2 client credentials flow를 사용하여 bearer token 가져오기.

    Returns:
        Access token 문자열
    """
    # 구성에서 scope 가져오기 (있는 경우)
    scope = CONFIG.get("client_info", {}).get("scope", "")

    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
    }

    # scope가 구성된 경우 추가
    if scope:
        data["scope"] = scope

    response = requests.post(
        TOKEN_ENDPOINT,
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data=data,
    )
    response.raise_for_status()
    return response.json()["access_token"]


def decode_token(access_token: str) -> Dict[str, Any]:
    """
    JWT token을 디코딩하여 claim 검사 (검증 없이).

    Args:
        access_token: JWT access token

    Returns:
        디코딩된 token payload (딕셔너리)
    """
    parts = access_token.split(".")
    if len(parts) != 3:
        raise ValueError("Invalid JWT token format")

    # payload 디코딩 (필요시 padding 추가)
    payload_encoded = parts[1]
    padding = 4 - len(payload_encoded) % 4
    if padding != 4:
        payload_encoded += "=" * padding

    return json.loads(base64.urlsafe_b64decode(payload_encoded))


def make_gateway_request(
    bearer_token: str, tool_name: str, arguments: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Amazon Bedrock AgentCore Gateway에 JSON-RPC 요청 수행.

    Args:
        bearer_token: OAuth2 access token
        tool_name: 호출할 tool 이름
        arguments: Tool 인자

    Returns:
        JSON-RPC 응답
    """
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments},
    }

    response = requests.post(
        GATEWAY_URL,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {bearer_token}",
            "Accept": "application/json",
        },
        json=payload,
    )
    response.raise_for_status()
    return response.json()


def analyze_response(result: Dict[str, Any]) -> str:
    """
    Gateway 응답을 분석하여 결과 판단.

    Returns:
        'ALLOWED', 'DENIED', 또는 'ERROR'
    """
    # JSON-RPC 오류 확인 (policy 거부는 특정 메시지와 함께 오류로 반환됨)
    if "error" in result:
        error_msg = result["error"].get("message", "").lower()
        # Policy 거부는 'tool call not allowed', 'access denied' 등의 오류 반환
        if any(
            phrase in error_msg
            for phrase in ["not allowed", "denied", "forbidden", "unauthorized action"]
        ):
            return "DENIED"
        return "ERROR"

    if "result" in result:
        # result가 오류를 나타내는지 확인 (일부 거부는 이 방식으로 반환됨)
        if result["result"].get("isError", False):
            content = result["result"].get("content", [])
            if content:
                text = (
                    content[0].get("text", "").lower()
                    if isinstance(content[0], dict)
                    else str(content[0]).lower()
                )
                if any(
                    phrase in text for phrase in ["not allowed", "denied", "forbidden"]
                ):
                    return "DENIED"
            return "DENIED"
        return "ALLOWED"

    return "UNKNOWN"


def display_test_result(expected: str, actual: str, description: str) -> bool:
    """
    테스트 결과를 포맷팅하여 표시.

    Returns:
        테스트 통과시 True, 그렇지 않으면 False
    """
    passed = expected == actual
    status = "✓ PASS" if passed else "✗ FAIL"
    print(f"\n{status}: {description}")
    print(f"   Expected: {expected}")
    print(f"   Actual: {actual}")
    return passed


print("✓ Helper functions defined")

---

## Part 3: Amazon Cognito Lambda Trigger 구성

JWT token에 커스텀 claim을 추가하려면 Amazon Cognito에서 Pre Token Generation AWS Lambda trigger를 구성해야 합니다.

### 중요 사항

- M2M(machine-to-machine) client credentials flow의 경우 AWS Lambda trigger 버전 **V3_0**을 **반드시** 사용해야 합니다
- V3_0은 Amazon Cognito **Essentials** 또는 **Plus** 티어가 필요합니다
- AWS Lambda 함수는 `department_name`, `groups` 등과 같은 커스텀 claim을 JWT token에 추가합니다
- 이러한 claim은 Cedar policy에서 **principal tag**가 됩니다

In [ ]:
def create_lambda_function(
    claims: Dict[str, Any], function_name: Optional[str] = None
) -> str:
    """
    Pre-token generation trigger를 위한 Lambda 함수 생성 또는 업데이트.

    Args:
        claims: Token에 추가할 claim 딕셔너리
        function_name: 선택적 커스텀 함수 이름

    Returns:
        Lambda 함수 ARN
    """
    if function_name is None:
        function_name = f"cognito-custom-claims-{USER_POOL_ID}"

    print(f"\nConfiguring Lambda Function: {function_name}")
    print("=" * 70)

    # 지정된 claim으로 Lambda 코드 생성
    claims_json = json.dumps(claims, indent=12)

    lambda_code = f'''
import json

def lambda_handler(event, context):
    """
    Cognito용 Pre-token generation V3 Lambda trigger.
    client_credentials를 포함한 모든 flow의 JWT token에 커스텀 claim 추가.
    """
    print(f"Event: {{json.dumps(event)}}")
    print(f"Trigger Source: {{event.get('triggerSource', 'unknown')}}")
    
    # Token에 커스텀 claim 추가
    event['response'] = {{
        'claimsAndScopeOverrideDetails': {{
            'accessTokenGeneration': {{
                'claimsToAddOrOverride': {claims_json}
            }},
            'idTokenGeneration': {{
                'claimsToAddOrOverride': {claims_json}
            }}
        }}
    }}
    
    print(f"Modified event: {{json.dumps(event)}}")
    return event
'''

    # 배포 패키지 생성
    with tempfile.NamedTemporaryFile(suffix=".zip", delete=False) as tmp_file:
        zip_path = tmp_file.name
        with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
            zipf.writestr("lambda_function.py", lambda_code)

    try:
        with open(zip_path, "rb") as f:
            zip_content = f.read()

        # 기존 함수 업데이트 시도
        try:
            lambda_client.update_function_code(
                FunctionName=function_name, ZipFile=zip_content
            )
            print("✓ Updated Lambda function code")
            response = lambda_client.get_function(FunctionName=function_name)
            return response["Configuration"]["FunctionArn"]

        except lambda_client.exceptions.ResourceNotFoundException:
            # IAM role과 함께 새 함수 생성
            role_name = f"{function_name}-role"
            role_arn = f"arn:aws:iam::{ACCOUNT_ID}:role/{role_name}"

            # 필요시 IAM role 생성
            try:
                iam_client.create_role(
                    RoleName=role_name,
                    AssumeRolePolicyDocument=json.dumps(
                        {
                            "Version": "2012-10-17",
                            "Statement": [
                                {
                                    "Effect": "Allow",
                                    "Principal": {"Service": "lambda.amazonaws.com"},
                                    "Action": "sts:AssumeRole",
                                }
                            ],
                        }
                    ),
                )
                iam_client.attach_role_policy(
                    RoleName=role_name,
                    PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
                )
                print(f"✓ Created IAM role: {role_name}")
                print("  Waiting for IAM role propagation...")
                time.sleep(10)  # IAM role 전파 대기
            except iam_client.exceptions.EntityAlreadyExistsException:
                print(f"  IAM role already exists: {role_name}")

            response = lambda_client.create_function(
                FunctionName=function_name,
                Runtime="python3.12",
                Role=role_arn,
                Handler="lambda_function.lambda_handler",
                Code={"ZipFile": zip_content},
                Timeout=30,
                MemorySize=128,
            )
            print("✓ Created Lambda function")
            return response["FunctionArn"]
    finally:
        os.remove(zip_path)


print("✓ Lambda creation function defined")

In [ ]:
def configure_cognito_trigger(lambda_arn: str) -> None:
    """
    Lambda trigger V3_0으로 Cognito User Pool 구성.

    Args:
        lambda_arn: Lambda 함수 ARN
    """
    print("\nConfiguring Cognito User Pool Trigger")
    print("=" * 70)

    # V3_0 trigger로 user pool 업데이트 (M2M에 필요)
    cognito_client.update_user_pool(
        UserPoolId=USER_POOL_ID,
        LambdaConfig={
            "PreTokenGenerationConfig": {
                "LambdaVersion": "V3_0",
                "LambdaArn": lambda_arn,
            }
        },
    )
    print("✓ User Pool trigger configured (V3_0)")
    print(f"  User Pool ID: {USER_POOL_ID}")
    print(f"  Lambda ARN: {lambda_arn}")

    # Cognito를 위한 Lambda 권한 추가
    try:
        lambda_client.add_permission(
            FunctionName=lambda_arn,
            StatementId=f"CognitoInvoke-{USER_POOL_ID}",
            Action="lambda:InvokeFunction",
            Principal="cognito-idp.amazonaws.com",
            SourceArn=f"arn:aws:cognito-idp:{REGION}:{ACCOUNT_ID}:userpool/{USER_POOL_ID}",
        )
        print("✓ Lambda permission added for Cognito")
    except lambda_client.exceptions.ResourceConflictException:
        print("  Lambda permission already exists")

    print("\n⚠️  IMPORTANT: V3_0 trigger requires Cognito Essentials or Plus tier")


print("✓ Cognito trigger function defined")

---

## Part 4: Policy Engine 함수

이러한 함수는 Policy for Amazon Bedrock AgentCore 서비스와 상호 작용하여 Cedar policy를 생성하고 관리합니다.

### JWT Claim을 위한 Cedar Policy 구문

JWT claim은 Cedar policy에서 **principal tag**를 통해 액세스됩니다:

| 패턴 | Cedar 구문 |
|---------|-------------|
| claim 존재 확인 | `principal.hasTag("claim_name")` |
| 정확한 일치 | `principal.getTag("claim_name") == "value"` |
| 패턴 일치 | `principal.getTag("claim_name") like "*value*"` |
| 입력 검증 | `context.input.field <= value` |

In [ ]:
def create_policy_engine(name: str) -> Optional[str]:
    """
    새 policy engine 생성.

    Args:
        name: Policy engine 이름

    Returns:
        성공시 Policy engine ID, 그렇지 않으면 None
    """
    print(f"\nCreating Policy Engine: {name}")
    print("=" * 70)

    try:
        import uuid

        response = policy_client.create_policy_engine(
            name=name,
            description=f"Policy engine created at {time.strftime('%Y-%m-%d %H:%M:%S')}",
            clientToken=str(uuid.uuid4()),
        )

        policy_engine_id = response["policyEngineId"]
        print("✓ Policy engine created")
        print(f"  Policy Engine ID: {policy_engine_id}")

        return policy_engine_id

    except ClientError as e:
        print(f"✗ Error creating policy engine: {e}")
        return None


def get_policy_engine(policy_engine_id: str) -> Optional[Dict[str, Any]]:
    """
    Policy engine 세부 정보 가져오기.
    """
    try:
        return policy_client.get_policy_engine(policyEngineId=policy_engine_id)
    except ClientError:
        return None


def wait_for_policy_engine_active(policy_engine_id: str, timeout: int = 300) -> bool:
    """
    Policy engine이 ACTIVE 상태에 도달할 때까지 대기.

    Args:
        policy_engine_id: Policy engine ID
        timeout: 최대 대기 시간 (초)

    Returns:
        활성화되면 True, 타임아웃 또는 실패시 False
    """
    print("\nWaiting for Policy Engine to become ACTIVE...")
    start_time = time.time()

    while time.time() - start_time < timeout:
        engine = get_policy_engine(policy_engine_id)
        if not engine:
            time.sleep(5)
            continue

        status = engine.get("status")
        print(f"  Status: {status}")

        if status == "ACTIVE":
            print("✓ Policy engine is ACTIVE")
            return True

        if status in ["CREATE_FAILED", "UPDATE_FAILED", "DELETE_FAILED"]:
            print(f"✗ Policy engine failed: {engine.get('statusReason', 'Unknown')}")
            return False

        time.sleep(5)

    print("✗ Timeout waiting for policy engine")
    return False


print("✓ Policy engine functions defined")

In [ ]:
def create_cedar_policy(
    policy_name: str, cedar_statement: str, description: str = ""
) -> Optional[str]:
    """
    Policy Engine에 Cedar policy 생성.

    Args:
        policy_name: Policy의 고유 이름
        cedar_statement: Cedar policy 문
        description: Policy 설명

    Returns:
        성공시 Policy ID, 그렇지 않으면 None
    """
    print(f"\nCreating Cedar Policy: {policy_name}")
    print("=" * 70)
    print("\nCedar Statement:")
    print("-" * 60)
    print(cedar_statement)
    print("-" * 60)

    try:
        response = policy_client.create_policy(
            policyEngineId=POLICY_ENGINE_ID,
            name=policy_name,
            description=description or f"Policy: {policy_name}",
            definition={"cedar": {"statement": cedar_statement}},
        )

        policy_id = response["policyId"]
        policy_status = response["status"]

        print("\n✓ Policy created successfully")
        print(f"  Policy ID: {policy_id}")
        print(f"  Status: {policy_status}")

        return policy_id

    except ClientError as e:
        error_code = e.response["Error"]["Code"]
        error_msg = e.response["Error"]["Message"]
        print(f"\n✗ Error creating policy: {error_code}")
        print(f"  {error_msg}")
        return None


def get_policy(policy_id: str) -> Optional[Dict[str, Any]]:
    """
    상태를 포함한 policy 세부 정보 가져오기.

    Args:
        policy_id: 가져올 Policy ID

    Returns:
        Policy 세부 정보 딕셔너리 또는 찾을 수 없으면 None
    """
    try:
        return policy_client.get_policy(
            policyEngineId=POLICY_ENGINE_ID, policyId=policy_id
        )
    except ClientError:
        return None


def wait_for_policy_active(policy_id: str, timeout: int = 60) -> bool:
    """
    Policy가 ACTIVE 상태에 도달할 때까지 대기.

    Args:
        policy_id: 확인할 Policy ID
        timeout: 최대 대기 시간 (초)

    Returns:
        Policy가 ACTIVE이면 True, 그렇지 않으면 False
    """
    start_time = time.time()

    while time.time() - start_time < timeout:
        policy = get_policy(policy_id)
        if not policy:
            print(f"  ⚠️  Policy not found: {policy_id}")
            return False

        status = policy.get("status")
        print(f"  Policy status: {status}")

        if status == "ACTIVE":
            return True

        if status in ["CREATE_FAILED", "UPDATE_FAILED"]:
            print(f"  ✗ Policy failed: {policy.get('statusReason', 'Unknown')}")
            return False

        time.sleep(3)

    print("  ✗ Timeout waiting for policy to become ACTIVE")
    return False


def delete_policy(policy_id: str) -> bool:
    """
    Policy Engine에서 policy 삭제.
    """
    try:
        policy_client.delete_policy(policyEngineId=POLICY_ENGINE_ID, policyId=policy_id)
        print(f"✓ Deleted policy: {policy_id}")
        return True
    except ClientError as e:
        print(f"⚠️  Could not delete policy {policy_id}: {e}")
        return False


def list_policies() -> List[Dict[str, Any]]:
    """
    Policy Engine의 모든 policy 나열.
    """
    try:
        response = policy_client.list_policies(policyEngineId=POLICY_ENGINE_ID)
        return response.get("policies", [])
    except ClientError:
        return []


print("✓ Cedar policy functions defined")

### Step 4.1: Policy Engine 존재 확인

policy engine이 존재하는지 확인하거나 필요한 경우 생성합니다.

In [ ]:
def ensure_policy_engine() -> str:
    """
    Policy engine이 존재하고 활성화되어 있는지 확인.
    필요시 생성하고 gateway_config.json 업데이트.

    Returns:
        Policy engine ID
    """
    global POLICY_ENGINE_ID, CONFIG

    print("\nEnsuring Policy Engine Exists")
    print("=" * 70)

    # 이미 policy engine ID가 있는지 확인
    if POLICY_ENGINE_ID:
        engine = get_policy_engine(POLICY_ENGINE_ID)
        if engine and engine.get("status") == "ACTIVE":
            print(f"✓ Using existing policy engine: {POLICY_ENGINE_ID}")
            return POLICY_ENGINE_ID

    # 기존 policy engine 나열
    try:
        response = policy_client.list_policy_engines()
        engines = response.get("policyEngines", [])

        for engine in engines:
            if engine.get("status") == "ACTIVE":
                POLICY_ENGINE_ID = engine["policyEngineId"]
                print(f"✓ Found existing ACTIVE policy engine: {POLICY_ENGINE_ID}")
                break
    except ClientError:
        pass

    # 필요시 새 policy engine 생성
    if not POLICY_ENGINE_ID:
        engine_name = f"PolicyEngine_{int(time.time())}"
        POLICY_ENGINE_ID = create_policy_engine(engine_name)

        if not POLICY_ENGINE_ID:
            raise RuntimeError("Failed to create policy engine")

        if not wait_for_policy_engine_active(POLICY_ENGINE_ID):
            raise RuntimeError("Policy engine did not become ACTIVE")

    # gateway_config.json에 저장
    CONFIG["policy_engine_id"] = POLICY_ENGINE_ID
    with open("gateway_config.json", "w") as f:
        json.dump(CONFIG, f, indent=2)
    print("✓ Saved policy_engine_id to gateway_config.json")

    return POLICY_ENGINE_ID


# Policy engine 존재 확인
POLICY_ENGINE_ID = ensure_policy_engine()

### Step 4.2: Gateway에 Policy Engine 연결

policy가 적용되려면 Policy Engine이 Gateway에 연결되어야 합니다. 이 단계는 Gateway에 이미 Policy Engine이 구성되어 있는지 확인하고, 그렇지 않은 경우 연결합니다.

In [ ]:
# 참고: gateway_control_client, get_gateway_details, wait_for_gateway_ready는
# Step 1.3에서 이미 정의됨


def attach_policy_engine_to_gateway(mode: str = "ENFORCE") -> bool:
    """
    Policy Engine을 Gateway에 연결 (아직 연결되지 않은 경우).

    Args:
        mode: Policy engine 모드 ('LOG_ONLY' 또는 'ENFORCE')

    Returns:
        성공하거나 이미 연결된 경우 True, 그렇지 않으면 False
    """
    print("\nAttaching Policy Engine to Gateway")
    print("=" * 70)

    # 현재 gateway 구성 가져오기
    gateway_config = get_gateway_details()

    # Policy engine이 이미 연결되어 있는지 확인
    existing_pe = gateway_config.get("policyEngineConfiguration", {})
    if existing_pe.get("arn"):
        print(f"✓ Policy Engine already attached: {existing_pe.get('arn')}")
        print(f"  Mode: {existing_pe.get('mode', 'N/A')}")
        return True

    # Policy engine ARN 가져오기
    engine = get_policy_engine(POLICY_ENGINE_ID)
    if not engine:
        print("✗ Could not get policy engine details")
        return False

    policy_engine_arn = engine.get("policyEngineArn")
    print(f"  Policy Engine ARN: {policy_engine_arn}")
    print(f"  Mode: {mode}")

    try:
        # boto3 client를 사용하여 gateway 업데이트
        gateway_control_client.update_gateway(
            gatewayIdentifier=GATEWAY_ID,
            name=gateway_config.get("name"),
            roleArn=gateway_config.get("roleArn"),
            protocolType=gateway_config.get("protocolType", "MCP"),
            authorizerType=gateway_config.get("authorizerType", "CUSTOM_JWT"),
            policyEngineConfiguration={"arn": policy_engine_arn, "mode": mode},
        )

        print("✓ Gateway update request accepted")
        print("\n⏳ Waiting for gateway to become READY...")

        if wait_for_gateway_ready():
            print("✓ Policy Engine attached successfully")
            return True
        else:
            print("✗ Gateway did not reach READY state")
            return False

    except ClientError as e:
        print(f"✗ Error updating gateway: {e}")
        return False


# Gateway에 policy engine 연결
attach_policy_engine_to_gateway(mode="ENFORCE")

---

## Part 5: 테스트 시나리오 1 - 부서 기반 접근 제어

이 시나리오에서는 **finance** 부서의 사용자만 요청을 허용하는 policy를 생성합니다.

### Cedar Policy 패턴

```cedar
permit(principal, action, resource)
when {
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance"
};
```

### Step 5.0: 기존 Policy 정리 (선택 사항)

새 policy를 생성하기 전에 기존 policy를 삭제하여 깨끗한 테스트 환경을 보장하는 것이 좋습니다. 이렇게 하면 이전 policy와 새 policy 간의 충돌을 방지할 수 있습니다.

In [ ]:
def cleanup_existing_policies(require_confirmation: bool = True) -> int:
    """
    Policy engine의 모든 기존 policy 삭제.

    Args:
        require_confirmation: True이면 삭제 전 사용자 확인 요청

    Returns:
        삭제된 policy 수
    """
    print("\n🧹 Checking for existing policies...")
    print("=" * 70)

    policies = list_policies()

    if not policies:
        print("✓ No existing policies found. Ready to proceed.")
        return 0

    print(f"\n⚠️  Found {len(policies)} existing policy/policies:")
    for p in policies:
        print(
            f"   - {p.get('name', 'unnamed')} (ID: {p.get('policyId')}, Status: {p.get('status')})"
        )

    if require_confirmation:
        print("\n" + "-" * 70)
        confirm = (
            input("Do you want to DELETE all existing policies? (yes/no): ")
            .strip()
            .lower()
        )
        if confirm != "yes":
            print("\n⏭️  Skipping cleanup. Existing policies will remain.")
            print("   Note: This may cause unexpected policy evaluation results.")
            return 0

    print("\n🗑️  Deleting existing policies...")
    deleted_count = 0
    for p in policies:
        policy_id = p.get("policyId")
        if policy_id and delete_policy(policy_id):
            deleted_count += 1

    print(f"\n✓ Deleted {deleted_count}/{len(policies)} policies")
    return deleted_count


# 마지막에 정리를 위해 생성된 policy 추적
CREATED_POLICIES = []

# 테스트 시작 전 기존 policy 정리
cleanup_existing_policies(require_confirmation=True)

### Step 5.1: Finance 부서 Claim으로 Lambda 구성

In [ ]:
print("=" * 70)
print("TEST SCENARIO 1: Department-Based Access Control")
print("=" * 70)

# department_name = "finance"로 Lambda 구성
claims_finance = {
    "department_name": "finance",
    "employee_level": "senior",
    "cost_center": "CC-1001",
}

lambda_arn = create_lambda_function(claims_finance)
configure_cognito_trigger(lambda_arn)

print("\n✓ Lambda configured with claims:")
print(json.dumps(claims_finance, indent=2))

### Step 5.2: Token에 커스텀 Claim이 포함되어 있는지 확인

In [ ]:
print("\nVerifying Token Claims")
print("=" * 70)

token = get_bearer_token()
claims = decode_token(token)

print("\nToken Claims (relevant):")
print(f"  department_name: {claims.get('department_name', 'NOT PRESENT')}")
print(f"  employee_level: {claims.get('employee_level', 'NOT PRESENT')}")
print(f"  cost_center: {claims.get('cost_center', 'NOT PRESENT')}")
print(f"  client_id: {claims.get('client_id', 'NOT PRESENT')}")

if claims.get("department_name") == "finance":
    print("\n✓ Custom claims verified in token")
else:
    print(
        "\n⚠️  Custom claims not found - Lambda trigger may not be configured correctly"
    )

### Step 5.3: 부서 검증을 위한 Cedar Policy 생성

In [ ]:
policy_name = f"dept_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance"
}};'''

print(f"Cedar statement:\n{cedar_statement}")

policy_id = create_cedar_policy(
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow requests only from finance department",
)

if policy_id:
    CREATED_POLICIES.append(policy_id)

    # Wait for policy to become ACTIVE (required before testing)
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_id):
        print("✓ Policy is ACTIVE and ready for testing")
    else:
        print("\n⚠️  Policy did not become ACTIVE. Tests may fail.")
        print("   Check the policy status in the AWS Console.")
else:
    print("\n✗ Failed to create policy. Cannot proceed with tests.")

### Step 5.4: Finance 부서로 테스트 (예상: ALLOWED)

In [ ]:
print("\n" + "=" * 70)
print("Test 1.1: Request with department_name='finance'")
print("=" * 70)

token = get_bearer_token()
result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-dept-finance"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "Finance department should be ALLOWED")

### Step 5.5: Engineering 부서로 테스트 (예상: DENIED)

In [ ]:
print("\n" + "=" * 70)
print("Test 1.2: Request with department_name='engineering'")
print("=" * 70)

# Update Lambda with different department
claims_engineering = {
    "department_name": "engineering",
    "employee_level": "senior",
    "cost_center": "CC-2001",
}

lambda_arn = create_lambda_function(claims_engineering)
print("\n✓ Lambda updated with department_name='engineering'")

# Wait for Lambda changes to propagate
print("\n⏳ Waiting for Lambda changes to propagate...")
time.sleep(5)

# Get new token and test
token = get_bearer_token()
claims = decode_token(token)
print(f"\nToken department_name: {claims.get('department_name')}")

result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-dept-engineering"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "Engineering department should be DENIED")

---

## Part 6: 테스트 시나리오 2 - 그룹 기반 접근 제어

이 시나리오에서는 **admins** 그룹에 속한 사용자만 요청을 허용하는 policy를 생성합니다.

### Cedar Policy 패턴

groups는 token에서 문자열로 직렬화되므로 패턴 일치를 위해 `like` 연산자를 사용합니다:

```cedar
permit(principal, action, resource)
when {
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*admins*"
};
```

In [ ]:
# 이전 policy 정리
print("=" * 70)
print("TEST SCENARIO 2: Groups-Based Access Control")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(pid)
CREATED_POLICIES.clear()

### Step 6.1: Groups Claim으로 Lambda 구성

In [ ]:
# "admins" 그룹을 포함하는 groups로 Lambda 구성
claims_with_admins = {
    "groups": ["admins", "developers", "team-alpha"],
    "department_name": "finance",
    "employee_level": "senior",
}

lambda_arn = create_lambda_function(claims_with_admins)
configure_cognito_trigger(lambda_arn)

print("\n✓ Lambda configured with groups:")
print(f"   groups: {claims_with_admins['groups']}")

### Step 6.2: Token에 Groups Claim이 포함되어 있는지 확인

In [ ]:
print("\nVerifying Token Claims")
print("=" * 70)

token = get_bearer_token()
claims = decode_token(token)

print("\nToken Claims (relevant):")
print(f"  groups: {claims.get('groups', 'NOT PRESENT')}")
print(f"  department_name: {claims.get('department_name', 'NOT PRESENT')}")

### Step 6.3: Groups 검증을 위한 Cedar Policy 생성

In [ ]:
policy_name = f"groups_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*admins*"
}};'''

print(f"Cedar statement:\n{cedar_statement}")

policy_id = create_cedar_policy(
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow requests only from users in admins group",
)

if policy_id:
    CREATED_POLICIES.append(policy_id)

    # Wait for policy to become ACTIVE (required before testing)
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_id):
        print("✓ Policy is ACTIVE and ready for testing")
    else:
        print("\n⚠️  Policy did not become ACTIVE. Tests may fail.")
else:
    print("\n✗ Failed to create policy. Cannot proceed with tests.")

### Step 6.4: Admins 그룹으로 테스트 (예상: ALLOWED)

In [ ]:
print("\n" + "=" * 70)
print("Test 2.1: Request with groups=['admins', 'developers', 'team-alpha']")
print("=" * 70)

token = get_bearer_token()
result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-groups-admins"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "User with 'admins' group should be ALLOWED")

### Step 6.5: Admins 그룹 없이 테스트 (예상: DENIED)

In [ ]:
print("\n" + "=" * 70)
print("Test 2.2: Request with groups=['developers', 'team-alpha']")
print("=" * 70)

# Update Lambda without admins group
claims_no_admins = {
    "groups": ["developers", "team-alpha"],
    "department_name": "finance",
    "employee_level": "senior",
}

lambda_arn = create_lambda_function(claims_no_admins)
print("\n✓ Lambda updated with groups (no admins):")

# Wait for Lambda changes to propagate
print("\n⏳ Waiting for Lambda changes to propagate...")
time.sleep(5)
print(f"   groups: {claims_no_admins['groups']}")

# Get new token and test
token = get_bearer_token()
claims = decode_token(token)
print(f"\nToken groups: {claims.get('groups')}")

result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-groups-no-admins"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "User without 'admins' group should be DENIED")

---

## Part 7: 테스트 시나리오 3 - Principal ID 기반 접근 제어

이 시나리오에서는 특정 principal(Client Credentials Flow의 `sub` claim으로 식별)의 요청만 허용하는 policy를 생성합니다.

### Cedar Policy 패턴

Policy for Amazon Bedrock AgentCore에서 principal은 평가할 수 있는 단일 속성인 `id`를 가집니다. Client Credentials Flow의 경우 이는 JWT token의 `sub` claim(`client_id`와 동일)에 해당합니다:

```cedar
permit(principal, action, resource)
when {
    principal.id == "your-client-id"
};
```

**참고**: `principal.hasTag()` 및 `principal.getTag()`를 통해 액세스되는 커스텀 claim과 달리 principal ID는 `principal.id`를 통해 직접 액세스됩니다.

In [ ]:
# Clean up previous policy
print("=" * 70)
print("TEST SCENARIO 3: Principal ID-Based Access Control")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(pid)
CREATED_POLICIES.clear()

### Step 7.1: Client ID 검증을 위한 Cedar Policy 생성

In [ ]:
policy_name = f"principal_id_policy_{int(time.time())}"

# In Client Credentials Flow, principal.id equals the 'sub' claim (which is the client_id)
cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.id == "{CLIENT_ID}"
}};'''

print(f"Cedar statement:\n{cedar_statement}")

policy_id = create_cedar_policy(
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description=f"Allow requests only from principal.id: {CLIENT_ID}",
)

if policy_id:
    CREATED_POLICIES.append(policy_id)

    # Wait for policy to become ACTIVE
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_id):
        print("✓ Policy is ACTIVE and ready for testing")
    else:
        print("\n⚠️  Policy did not become ACTIVE. Tests may fail.")

### Step 7.2: 일치하는 Principal ID로 테스트 (예상: ALLOWED)

In [ ]:
print("\n" + "=" * 70)
print(f"Test 3.1: Request with principal.id='{CLIENT_ID}'")
print("=" * 70)

token = get_bearer_token()
claims = decode_token(token)
# In Client Credentials Flow, 'sub' claim equals client_id and maps to principal.id
print(f"\nToken 'sub' claim (principal.id): {claims.get('sub')}")
print(f"Token 'client_id' claim: {claims.get('client_id')}")

result = make_gateway_request(
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 500, "orderId": "test-principal-id"},
)

print("\nRequest: RefundToolTarget___refund(amount=500)")
print("\nResponse:")
print(json.dumps(result, indent=2))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "Matching principal.id should be ALLOWED")

print("\n💡 Note: To test DENY scenario, you would need a different")
print("   Amazon Cognito app client with a different client_id (sub)")

---

## Part 8: 고급 패턴

### 여러 조건 결합

더 복잡한 접근 제어 시나리오를 위해 단일 policy에서 여러 조건을 결합할 수 있습니다.

In [ ]:
# Example: Combined policy (department AND amount limit)
print("\nAdvanced Pattern: Combined Conditions")
print("=" * 70)

combined_cedar = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("department_name") &&
    principal.getTag("department_name") == "finance" &&
    context.input.amount <= 1000
}};'''

print("Cedar Policy with Combined Conditions:")
print("-" * 60)
print(combined_cedar)
print("-" * 60)
print("\nThis policy allows requests when:")
print("  ✓ User is in finance department")
print("  ✓ AND refund amount is <= $1000")

### `like` 연산자를 사용한 패턴 일치

`like` 연산자는 유연한 일치를 위해 와일드카드를 지원합니다:

| 패턴 | 일치 |
|---------|--------|
| `"*admin*"` | 어디든 "admin" 포함 |
| `"admin*"` | "admin"으로 시작 |
| `"*admin"` | "admin"으로 끝남 |
| `"team-*"` | "team-"으로 시작 |

In [ ]:
# Example: Pattern matching for team-based access
print("\nAdvanced Pattern: Team-Based Access with Wildcards")
print("=" * 70)

team_cedar = f'''permit(principal,
    action == AgentCore::Action::"RefundToolTarget___refund",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    principal.hasTag("groups") &&
    principal.getTag("groups") like "*team-finance*"
}};'''

print("Cedar Policy with Pattern Matching:")
print("-" * 60)
print(team_cedar)
print("-" * 60)
print("\nThis policy allows requests when:")
print("  ✓ User's groups contain 'team-finance'")
print("  ✓ Matches: ['team-finance', 'developers']")
print("  ✓ Matches: ['admins', 'team-finance-leads']")

---

## Part 9: 모범 사례

### Policy 설계 모범 사례

1. **특정 action 사용** - 와일드카드보다는 특정 tool action을 대상으로 지정
2. **항상 claim 존재 확인** - 오류를 방지하기 위해 `getTag()` 전에 `hasTag()` 사용
3. **패턴 일치를 신중하게 사용** - `like "*value*"`는 의도하지 않은 문자열과 일치할 수 있음
4. **ALLOW와 DENY 모두 테스트** - policy가 양방향으로 작동하는지 확인
5. **policy 문서화** - 설명적인 이름과 설명 사용

### Amazon Cognito 구성 모범 사례

1. **V3_0 trigger 사용** - M2M client credentials flow에 필요
2. **Essentials 티어로 업그레이드** - V3_0은 Amazon Cognito Essentials 또는 Plus 필요
3. **token claim 테스트** - policy를 생성하기 전에 항상 claim이 token에 나타나는지 확인
4. **배열을 신중하게 처리** - 배열은 JWT claim에서 문자열로 직렬화됨

### 피해야 할 일반적인 함정

- ❌ M2M flow에서 V1_0 또는 V2_0 trigger 사용 (claim이 추가되지 않음)
- ❌ `getTag()` 전에 `hasTag()` 확인을 잊음
- ❌ 배열에 패턴 일치(`like`)가 필요할 때 정확한 일치(`==`) 사용
- ❌ 테스트하기 전에 policy가 ACTIVE 상태가 될 때까지 기다리지 않음
- ❌ 적절한 정리 전략 없이 policy 생성

### 필요한 AWS IAM 권한

#### Policy 관리용

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "policy-registry:CreatePolicyEngine",
        "policy-registry:GetPolicyEngine",
        "policy-registry:ListPolicyEngines",
        "policy-registry:CreatePolicy",
        "policy-registry:DeletePolicy",
        "policy-registry:ListPolicies"
      ],
      "Resource": "*"
    }
  ]
}
```

#### Amazon Cognito AWS Lambda Trigger용

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "cognito-idp:UpdateUserPool",
        "lambda:CreateFunction",
        "lambda:UpdateFunctionCode",
        "lambda:AddPermission",
        "iam:CreateRole",
        "iam:AttachRolePolicy",
        "iam:PassRole"
      ],
      "Resource": "*"
    }
  ]
}
```

---

## Part 10: 정리

이 튜토리얼 중에 생성된 모든 policy를 삭제합니다.

In [ ]:
print("=" * 70)
print("CLEANUP")
print("=" * 70)

print(f"\nDeleting {len(CREATED_POLICIES)} policies...")
for pid in CREATED_POLICIES:
    delete_policy(pid)

CREATED_POLICIES.clear()
print("\n✓ Cleanup complete")

### 선택 사항: 모든 Policy 삭제

policy engine의 모든 policy를 정리하는 데 사용합니다 (주의해서 사용).

In [ ]:
# Uncomment to delete ALL policies in the policy engine
# WARNING: This will delete all policies, not just those created in this tutorial

# print("Deleting ALL policies...")
# policies = list_policies()
# for policy in policies:
#     policy_id = policy.get('policyId')
#     if policy_id:
#         delete_policy(policy_id)
# print("✓ All policies deleted")

---

## 결론

축하합니다! Policy for Amazon Bedrock AgentCore 튜토리얼을 완료했습니다. 다음을 학습했습니다:

✅ JWT token에 커스텀 claim을 추가하기 위한 Amazon Cognito AWS Lambda trigger 구성  
✅ principal tag를 통해 JWT claim을 검증하는 Cedar policy 생성  
✅ 부서 기반 접근 제어 구현  
✅ 패턴 일치를 사용한 그룹 기반 접근 제어 구현  
✅ Principal ID 기반 접근 제어 구현  
✅ 복잡한 접근 제어 시나리오를 위한 여러 조건 결합  

### 주요 Cedar 구문 패턴

| Claim 유형 | Cedar 구문 |
|------------|-------------|
| 문자열 (정확한 일치) | `principal.getTag("claim") == "value"` |
| 문자열 (포함) | `principal.getTag("claim") like "*value*"` |
| 배열 (포함) | `principal.getTag("claim") like "*value*"` |
| 입력 검증 | `context.input.field <= value` |

### 다음 단계

1. **프로덕션에 구현** - 이러한 패턴을 Amazon Bedrock AgentCore 배포에 적용
2. **policy 커스터마이징** - 특정 접근 제어 요구 사항에 맞게 policy 조정
3. **모니터링 추가** - policy 거부에 대한 Amazon CloudWatch 알람 설정
4. **반복 및 개선** - 실제 사용을 기반으로 policy 개선

---

**Author**: AWS  
**License**: MIT-0  
**Last Updated**: 2025